In [1]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
def encoding_base_block(input_tensor,filters,kernal_size,activation='relu',batch_normalization=True):
    x = inputs_tensor
    x = layers.Conv2D(filters=filters,kernal_size=kernal_size,padding='same',kernal_initializer='he_normal')(x)
    
    if batch_normalization:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(filters=filters,kernal_size=kernal_size,padding='same',kernal_initializer='he_normal')(x)
    
    if batch_normalization:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    return x


    
    


In [3]:
def generate_unet(image_tensor,n_filter=16,drop_out=0.1,batch_normalization =True):
    
    x = image_tensor
    
    # encoding/reduction section
    
    conv1 = encoding_base_block(input_tensor = x,filters=n_filters*1,kernal_size=(2,2),batch_normalization=batch_normalization)
    pool1 = layers.MaxPool2D((2, 2))(conv1)
    pool1 = layers.Dropout(dropout)(pool1)
    
    conv2 = encoding_base_block(input_tensor = pool1,filters=n_filter*2,kernal_size=(2,2),batch_normalization=batch_normalization)
    pool2 = layers.MaxPool2D((2,2))(conv2)
    pool2 = layers.Dropout(dropout)(pool2)
    
    conv3 = encoding_base_block(input_tensor = pool2,filters=n_filter*4,kernal_size=(2,2),batch_normalization=batch_normalization)
    pool3 = layers.MaxPool2D((2,2))(conv3)
    pool3 = layers.Dropout(dropout)(pool3)
    
    conv4 = encoding_base_block(input_tensor = pool3,filters=n_filter*8,kernal_size=(2,2),batch_normalization=batch_normalization)
    pool4 = layers.MaxPool2D((2,2))(conv4)
    pool4 = layers.Dropout(dropout)(pool4)
    
    conv5 = encoding_base_block(input_tensor=pool4,filters=n_filter*16,kernal_size=(2,2),batch_normalization=batch_normalization)
    
    up6 = layers.Conv2DTranspose(n_filters*8,(3,3),strides=(2,2),padding='same')(conv5)
    up6 = layers.concatenate([up6,conv4])
    up6 = layers.Dropout(drop_out)(up6)
    conv6 = encoding_base_block(input_tensor=up6,filters=n_filter*8,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    up7 = layers.Conv2DTranspose(n_filters*4,(3,3),strides=(2,2),padding='same')(conv6)
    up7 = layers.concatenate([up7,conv3])
    up7 = layers.Dropout(drop_out)(up7)
    conv7 = encoding_base_block(input_tensor=up7,filters=n_filter*4,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    up8 = layers.Conv2DTranspose(n_filters*2,(3,3),strides=(2,2),padding='same')(conv7)
    up8 = layers.concatenate([up8,conv2])
    up8 = layers.Dropout(drop_out)(up8)
    conv8 = encoding_base_block(input_tensor=up8,filters=n_filter*2,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    up9 = layers.Conv2DTranspose(n_filters*2,(3,3),strides=(2,2),padding='same')(conv8)
    up9 = layers.concatenate([up9,conv1])
    up9 = layers.Dropout(drop_out)(up9)
    conv9 = encoding_base_block(input_tensor=up9,filters=n_filter,kernal_size=(3,3),batch_normalization=batch_normalization)
    
    
    outputs = layers.Conv2D(1,(1,1),activation='sigmoid')(conv9)
    
    
    model = Model(inputs = [image_tensor],outputs = [outputs])
    
    return model

In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf


from IPython.display import clear_output
import matplotlib.pyplot as plt

In [10]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

In [11]:
@tf.function
def load_image_train(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [12]:
def load_image_test(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
from PIL import Image

In [ ]:
im1 = Image.open('C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\trimaps\\Abyssinian_1.png')
im2 = Image.open('C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\images\\Abyssinian_1.jpg')

In [ ]:
im1.show()

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [19]:
columns = ['x_col','y_col']
df = pd.DataFrame(columns=columns)

trainval_path = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation\\annotations\\trainval.txt'
f = open(trainval_path,'r')
image_names = []
for i in f:
    image_name = i.split(' ')[0]
    image_names.append(['images\\'+ image_name + '.jpg','annotations\\trimaps\\' + image_name+'.png'])
    
    
df = pd.DataFrame(image_names,columns = columns)
len(df)

3680

In [18]:
directory = 'C:\\Users\\tayyabm\\Downloads\\oxford_cats_segmentation'
train_image_generator = ImageDataGenerator().flow_from_dataframe(dataframe=df,x_col='x_col',y_col='y_col',directory=directory,class_mode='raw')
train_mask_generator = ImageDataGenerator().flow_from_dataframe(dataframe=df,x_col='y_col',y_col='y_col',directory=directory,class_mode='raw')

Found 3680 validated image filenames.


'raw'

### UNET with Quantization